데코레이터가 무엇인지 어떻게 동작하는지 그리고 어떻게 구현되는지 살펴본다. 실제 데코레이터가 디자인을 개선하는데 어떻게 유용하게 사용될 수 있는지도 살펴본다.

이렇게 배운 내용을 토대로 이전 장에서 배운 소프트웨어 디자인의 모범 사례들을 다시 살펴보고 데코레이터가 각 원칙을 준수하는데 어떻게 도움이 되는지 살펴본다.

* 파이썬에서 데코레이터가 동작하는 방식 이해
* 함수와 크랠스에 적용되는 데코레이터를 구현하는 방법
* 일반적인 실수를 피하여 데코레이터를 효과적으로 구현하는 방법
* 데코레이터를 활용한 코드 중복을 회피(DRY 원칙)
* 데코레이터를 활용한 관심사의 분리 
* 좋은 데코레이터 사례 
* 데코레이터가 좋은 선택이 될 수 있는 일반적인 상황, 관용구, 패턴

## 파이썬의 데코레이터

파이썬에서 데코레이터는 오래 전에(PRP-318)에서 함수와 메서드의 기능을 쉽게 수정하기 위한 수단으로 소갱되었다. 

classmethod나 staticmethod 같은 함수가 원래 메서드의 정의를 변형하는데 사용되고 있었기 때문에 고안된 수단인데 이런 방법은 추가 코드가 필요하고 함수의 원래 정의를 수정해야만 했다.

다시 말해서 함수에 변형을 할 때마다 modifier 함수를 사용하여 함수를 호출한 다음 함수를 처음 정의한 것과 같은 이름으로 재할당해야 했다.

예를 들어 original 함수에 그 기능을 약간 수정한 modifier라고 하는 함수가 있는 경우 다음과 같이 작성해야 했다.

In [3]:
def original():
    return None

def modifier(ori):
    return ori()

original = modifier(original)

함수를 어떻게 동일한 이름으로 다시 할당하는지 확인해보자. 이것은 혼랍스럽고 오류가 발생하기 쉽고 번거롭다. 이런 이유로 새로운 구문이 추가되었다.

In [5]:
@modifier
def original():
    return None

즉, 데코레이터는 데코레이터 이후에 나오는 것을 데코레이터 첫 번째 파라미터로 하고 데코레이터의 결과 값을 반환하게 하는 문법적 설탕(타이필 수고 또는 읽기 쉽게 하기 위해 다른 표현으로 코딩할 수 있게 해주는 기능)일 뿐이다.

이번 예제에서 말하는 modifier는 파이썬 용어로 데코레이터라 하고, original을 데코레이팅된 함수 또는 래핑된 객체라 한다.

원래 함수와 메서드를 위해 고안되었지만 실제로는 어떤 종류의 객체에도 적용이 가능하기 때문에 여기서는 함수와 메서드, 제너레이터, 클래스에 데코레이터를 적용하는 방법을 살펴본다.

데코레이터는 래핑된 함수의 기능을 수정하고 확장하기 때문에 정확한 이름이지만 "데코레이터 디자인 패턴"과 혼동하면 안 된다.

### 함수 데코레이터

파이썬에서 데코레이터를 사용하여 기능을 변경하는 가장 간단한 방법은 함수에 적용하는 것이다. 함수에 데코레이터를 사용하면 어떤 종류의 로직이라도 적용할 수 있다. 파라미터의 유효성을 검사하거나 사전조건을 검사하거나, 기능 전체를 새롭게 정의할 수도 있고, 서명을 변경할 수도 있고, 원래 함수의 결과를 캐시하는 드으이 작업을 모두 할 수 있다.

예를 들어, 다음과 같인 도메인의 특정 예외에 대해서 특정 획수만큼 재시도하는 데코레이터를 만들어 볼 수 있다.

In [ ]:
class ControlledException(Exception):
    """도메인에서 발생하는 일반적인 예외"""

def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised
    return wrapped